# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv
from cassandra.cluster import Cluster

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/marco/Documents/data-engineer/data modeling/NO-SQL


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
print("# Number of elements in the list ",len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
print('Sample Data: \n',full_data_rows_list[0])

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


# Number of elements in the list  4102
Sample Data: 
 ['Mudhoney', 'Logged In', 'Aleena', 'F', '10', 'Kirby', '231.57506', 'paid', 'Waterloo-Cedar Falls, IA', 'PUT', 'NextSong', '1.54102E+12', '637', 'Get Into Yours', '200', '1.54233E+12', '44']


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

In [5]:
['artist','firstName','gender','itemInSession','lastName','length','level','location','sessionId','song','userId']

['artist',
 'firstName',
 'gender',
 'itemInSession',
 'lastName',
 'length',
 'level',
 'location',
 'sessionId',
 'song',
 'userId']

#### Creating a Cluster

In [6]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [7]:
# TO-DO: Create a Keyspace 
try:
    session.execute('''
    CREATE KEYSPACE IF NOT EXISTS spartifydb
    WITH REPLICATION = 
    {'class' : 'SimpleStrategy', 'replication_factor' : 1}
    ''')
    print('Initilize DB!')
    
except Exception as e:
    print(e)

Initilize DB!


#### Set Keyspace

In [8]:
# TO-DO: Set KEYSPACE to the keyspace specified above

try:
    session.set_keyspace('spartifydb')
    print('Key space set !!!')
except Exception as e:
    print(e)


Key space set !!!


### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [12]:
## TO-DO: Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4
create_query1 = 'CREATE TABLE IF NOT EXISTS song_insession_initem'
create_query1 += '''(session_id INT, item_in_session INT, artist TEXT, song_title TEXT, song_length FLOAT, PRIMARY KEY(session_id, item_in_session))'''

try:
    session.execute(create_query1)
    print('Table Made !!')
except Exception as e:
    print(e)

    
select_query1 = '''
SELECT artist,song_title,song_length FROM song_insession_initem
WHERE session_id = %s AND item_in_session=%s'''

                    

Table Made !!


In [13]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## TO-DO: Assign the INSERT statements into the `query` variable
        query = "INSERT INTO song_insession_initem(session_id, item_in_session, artist, song_title, song_length)"
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        ## TO-DO: Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

#### Do a SELECT to verify that the data have been inserted into each table

In [15]:
## TO-DO: Add in the SELECT statement to verify the data was entered into the table
try:
    rows = session.execute(select_query1,(338,4))
    print('Answer !!!')
except Exception as e:
    print(e)
    
for row in rows:
    print("Artist: "+row.artist, "\nSong Title: "+row.song_title, "\nSong Length: "+str(row.song_length))

Answer !!!
Artist: Faithless 
Song Title: Music Matters (Mark Knight Dub) 
Song Length: 495.30731201171875


### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

In [16]:
## TO-DO: Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182

create_query2 = 'CREATE TABLE IF NOT EXISTS artist_song'
create_query2 += '''(user_id INT, session_id INT,item_in_session INT, artist TEXT, song_title TEXT, user_first_name TEXT, 
                                        user_last_name TEXT, PRIMARY KEY(user_id,session_id, item_in_session))'''

try:
    session.execute(create_query2)
    print('Table Made !!')
except Exception as e:
    print(e)

    
select_query1 = '''
SELECT artist , song_title, user_first_name, user_last_name FROM artist_song
WHERE user_id = %s AND session_id = %s'''
                    

Table Made !!


In [20]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## TO-DO: Assign the INSERT statements into the `query` variable
        query1 = "INSERT INTO artist_song(user_id, session_id,item_in_session, artist, song_title, user_first_name, user_last_name)"
        query1 = query1 + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
        ## TO-DO: Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query1, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))
                    

In [22]:
try:
    rows = session.execute(select_query1,(10,182))
    print('Answer !!!')
except Exception as e:
    print(e)
    
for row in rows:
    print("Artist: "+row.artist, "\nSong Title: "+ row.song_title,"\nFirst Name: "+row.user_first_name, "\nLast Name: "+row.user_last_name)

Answer !!!
Artist: Down To The Bone 
Song Title: Keep On Keepin' On 
First Name: Sylvie 
Last Name: Cruz
Artist: Three Drives 
Song Title: Greece 2000 
First Name: Sylvie 
Last Name: Cruz
Artist: Sebastien Tellier 
Song Title: Kilometer 
First Name: Sylvie 
Last Name: Cruz
Artist: Lonnie Gordon 
Song Title: Catch You Baby (Steve Pitron & Max Sanna Radio Edit) 
First Name: Sylvie 
Last Name: Cruz


In [28]:
## TO-DO: Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

create_query3 = 'CREATE TABLE IF NOT EXISTS user_by_song'
create_query3 += '''(song_title TEXT, user_id INT, user_first_name TEXT, user_last_name TEXT, PRIMARY KEY(song_title,user_id))'''

try:
    session.execute(create_query3)
    print('Table Made !!')
except Exception as e:
    print(e)

    
select_query2 = '''
SELECT song_title, user_first_name, user_last_name FROM user_by_song
WHERE song_title = %s'''
                    
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## TO-DO: Assign the INSERT statements into the `query` variable
        query2 = "INSERT INTO user_by_song(song_title,user_id,user_first_name, user_last_name)"
        query2 = query2 + "VALUES (%s, %s, %s, %s)"
        
        session.execute(query2, (line[9],int(line[10]),line[1], line[4]))

try:
    rows = session.execute(select_query2,('All Hands Against His Own',)) # because we did'not use all primary key 
    print('Answer !!!')
except Exception as e:
    print(e)
    
for row in rows:
    print("\nFirst Name: "+row.user_first_name, "\nLast Name: "+row.user_last_name)



Table Made !!
Answer !!!

First Name: Jacqueline 
Last Name: Lynch

First Name: Tegan 
Last Name: Levine

First Name: Sara 
Last Name: Johnson


### Drop the tables before closing out the sessions

In [4]:
## TO-DO: Drop the table before closing out the sessions

In [29]:
session.execute("DROP TABLE IF EXISTS song_insession_initem")
session.execute("DROP TABLE IF EXISTS artist_song")
session.execute("DROP TABLE IF EXISTS user_by_song")

### Close the session and cluster connection¶

In [30]:
session.shutdown()
cluster.shutdown()